# 신경망으로 챗봇 만들기

이전 노트북에서는 코사인 유사성을 가진 챗봇을 만드는 방법을 배웠습니다. 이제는 신경망을 이용해 어떻게 만들 수 있는지 살펴보겠습니다!

훈련 데이터를 만들고 신경망을 훈련시킨 다음 훈련된 모델을 사용하여 챗봇을 만들 것입니다.

먼저, 필수 라이브러리를 설치할 것입니다. 라이브러리가 설치되지 않은 경우에만 아래 몇 개의 블록을 주석 해제하십시오.

In [1]:
#!pip install numpy scipy
#!pip install scikit-learn
#!pip install pillow
#!pip install h5py

In [2]:
#!pip install tensorflow

In [3]:
#!pip install tensorflow-gpu

In [4]:
#!pip install keras

# 1. 라이브러리 설치

우선 이 신경망 구동 챗봇에 필요한 라이브러리를 설치하겠습니다.
Keras는 백엔드에서 텐서플로우(다른 하위 레벨 기계 학습 라이브러리) 를 활용하는 기계 학습 라이브러리입니다. 이렇게 하면 우리의 목적을 위해 심층 신경망을 쉽게 배포할 수 있습니다.

In [2]:
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras.layers import Dense
 
from numpy import argmax
import numpy as np
import re

/Users/hong/venvs/nlp_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# 2. 입력 훈련 데이터

먼저 챗봇에 대한 다음 교육 데이터를 포함하겠습니다.:
1. X는 사용자가 입력할 수 있는 다양한 입력을 나타냅니다.
2. Y는 입력의 의도를 나타냅니다.

In [25]:
X = ['Hi',
     'Hello',
     'How are you?',
     'I am making',
     'making',
     'working',
     'studying',
     'see you later',
     'bye',
     'goodbye']

In [26]:
print(len(X))

10


In [27]:
Y = ['greeting',
     'greeting',
     'greeting',
     'busy',
     'busy',
     'busy',
     'busy',
     'bye',
     'bye',
     'bye']

In [28]:
print(len(Y))

10


비슷한 의도를 가진 여러 개의 다른 문장들이 있다는 것을 주목하십시오. 여기에서는 3개의 의도(greeting, busy, bye)만 있지만 프로젝트에 원하는 만큼 추가할 수 있습니다.

이것은 챗봇이 작동하는 방식입니다:
1. 입력 문장으로부터, 우리는 훈련된 AI 모델을 사용하여 의도를 확인할 것이다.
2. 각 의도에 대해, 우리는 준비 된 응답을 가지고있다.

예를 들어, 입력의 의도가 인사말에 대한 것임을 확인하면 챗봇에 '안녕하세요'또는 '어떻게 지내십니까?'와 같은 인사말로 응답하도록 요청할 수 있습니다.

우리는 기계 학습을 사용하여 입력 문장을 다른 의도로 분류 할 수있는 모델을 만들 것입니다. 
다음과 같이 만듭니다:

1. 문장과 그 의도에 대한 목록을 포함하고 있는 훈련 데이터(위의 X및  Y)를 작성한다.
2. 훈련 데이터를 사용하여 분류기를 훈련한다. 
3. 입력 문장을 벡터화하고 분류기를 사용하여 의도를 결정한다. 

# 3. 텍스트 처리

평소와 같이 텍스트 처리부터 시작합니다. 그 과정을 기억하십니까?

## 3.1 알파벳과 숫자가 아닌 문자 제거

In [31]:
def remove_non_alpha_numeric_characters(sentence):
    new_sentence = ''
    for alphabet in sentence:
        if alphabet.isalpha() or alphabet == ' ':
            new_sentence += alphabet
    return new_sentence

In [32]:
def preprocess_data(X):
    X = [data_point.lower() for data_point in X]
    X = [remove_non_alpha_numeric_characters(
        sentence) for sentence in X]
    X = [data_point.strip() for data_point in X]
    X = [re.sub(' +', ' ',
                data_point) for data_point in X]
    return X

In [33]:
X = preprocess_data(X)

vocabulary = set()
for data_point in X:
    for word in data_point.split(' '):
        vocabulary.add(word)

vocabulary = list(vocabulary)

## 문서 벡터 생성

In [34]:
X_encoded = []

def encode_sentence(sentence):
    sentence = preprocess_data([sentence])[0]
    sentence_encoded = [0] * len(vocabulary)
    for i in range(len(vocabulary)):
        if vocabulary[i] in sentence.split(' '):
            sentence_encoded[i] = 1
    return sentence_encoded

X_encoded = [encode_sentence(sentence) for sentence in X]

In [35]:
classes = list(set(Y))

Y_encoded = []
for data_point in Y:
    data_point_encoded = [0] * len(classes)
    for i in range(len(classes)):
        if classes[i] == data_point:
            data_point_encoded[i] = 1
    Y_encoded.append(data_point_encoded)

# 4. 훈련 데이터 및 테스트 데이터 생성

In [36]:
X_train = X_encoded
y_train = Y_encoded
X_test = X_encoded
y_test = Y_encoded

훈련 및 테스트 데이터에 사용하는 데이터 출력 및 확인

In [37]:
print (y_test)

[[0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0]]


In [38]:
print(len(X_train))

10


In [39]:
y_train

[[0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0]]

y_train은 무엇을 의미합니까? 위에 표시된 배열을 이해합니까?

# 5. 모델 훈련

이제 훈련 데이터를 이용해 신경망을 훈련시키겠습니다.

In [40]:
model = Sequential()
model.add(Dense(units=64, activation='sigmoid',
                input_dim=len(X_train[0])))
model.add(Dense(units=len(y_train[0]), activation='softmax'))
model.compile(loss=categorical_crossentropy,
              optimizer=SGD(lr=0.01,
                            momentum=0.9, nesterov=True))
model.fit(np.array(X_train), np.array(y_train), epochs=100, batch_size=16)

/usr/local/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/100
1/1 [==============================] - 0s 312ms/step - loss: 1.1653
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 1.1467
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 1.1250
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 1.1040
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0866
Epoch 6/100
1/1 [==============================] - 0s 3ms/step - loss: 1.0743
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0673
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0645
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0647
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0662
Epoch 11/100
1/1 [==============================] - 0s 4ms/step - loss: 1.0676
Epoch 12/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0679
Epoch 13/100
1/1 [==============================] - 0s 3ms/

## 예측 목록 표시

In [41]:
predictions = [argmax(pred) for pred in model.predict(np.array(X_test))]

# 모델 평가

이제 모델을 평가해 봅시다. 모델에 의한 예측과 테스트 데이터를 비교할 것입니다:

In [42]:
correct = 0
for i in range(len(predictions)):
    if predictions[i] == argmax(y_test[i]):
        correct += 1

print ("Correct:", correct)
print ("Total:", len(predictions))

Correct: 8
Total: 10


# 챗봇 테스트

이제 챗봇을 테스트해 보겠습니다! 문장을 입력한 다음 신경망에서 예측되는 클래스를 확인합니다:

In [44]:
while True:
    print ("Enter a sentence")
    sentence = input()
    prediction= model.predict(np.array([encode_sentence(sentence)]))
    print (classes[argmax(prediction)])

Enter a sentence
goodbye
bye
Enter a sentence
bye
bye
Enter a sentence
How are you?
greeting
Enter a sentence


KeyboardInterrupt: Interrupted by user

ㅇㄹ


챗봇을 멈출 수 없다는 것을 알고 있습니까? 나중에 종료 명령을 추가해야 합니다(이전 노트를 참조하여 수행 방법을 확인하십시오.).

일단은 위의 중지 버튼(인터럽트 버튼)을 눌러서 챗봇을 중지하면 됩니다.

시도해 보세요. 정지 버튼을 누르고 상자에 뭔가를 입력해 보세요.

# 도전과제

우리는 성공적으로 대화 의도에 우리의 입력을 매핑하는 신경망을 사용했습니다. 
여러분의 과제는 대화 의도를 챗봇이 말하는 특정 응답과 연결하는 것입니다. 
예를 들어, 만약 대화의 목적이 '인사' 라면, 여러분의 챗봇도 인사말을 하도록 하세요!

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# NLTK 데이터 다운로드
# 처음 한 번만 실행하면 됩니다.
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/hong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/hong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
df = pd.read_csv('[Dataset]_Module25_disasters_social_media.csv')
df.head()

,Unnamed: 0,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one,choose_one:confidence,choose_one_gold,keyword,location,text,tweetid,userid
0,0,778243823,True,golden,156,NaN,Relevant,1.0000,Relevant,NaN,NaN,Just happened a terrible car crash,1.0,NaN
1,1,778243824,True,golden,152,NaN,Relevant,1.0000,Relevant,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,13.0,NaN
2,2,778243825,True,golden,137,NaN,Relevant,1.0000,Relevant,NaN,NaN,"Heard about #earthquake is different cities, s...",14.0,NaN
3,3,778243826,True,golden,136,NaN,Relevant,0.9603,Relevant,NaN,NaN,"there is a forest fire at spot pond, geese are...",15.0,NaN
4,4,778243827,True,golden,138,NaN,Relevant,1.0000,Relevant,NaN,NaN,Forest fire near La Ronge Sask. Canada,16.0,NaN


In [41]:
df_choose = df[df['choose_one'] != "Can't Decide"][['text', 'choose_one']]
df_choose.head()

,text,choose_one
0,Just happened a terrible car crash,Relevant
1,Our Deeds are the Reason of this #earthquake M...,Relevant
2,"Heard about #earthquake is different cities, s...",Relevant
3,"there is a forest fire at spot pond, geese are...",Relevant
4,Forest fire near La Ronge Sask. Canada,Relevant


In [42]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))


def preprocess_text(text):
    # lower
    text = text.lower()

    # punctuation removal
    text = re.sub(r"[^a-z\s]", "", text)

    # unwanted space removal
    text = re.sub(r"\s+", " ", text).strip()

    # lemmatize
    words = text.split()  # split makes list
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return ' '.join(words)

df_choose['text'] = df_choose['text'].apply(preprocess_text)

In [66]:
# label encoding

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_choose['choose_one'])

# 인코딩된 레이블과 원래 레이블을 출력하여 확인
print("원본 레이블:\n", df_choose['choose_one'])
print("인코딩된 레이블:\n", y_encoded)

# 인코딩된 숫자를 다시 원래 레이블로 변환할 수도 있습니다.
y_decoded = label_encoder.inverse_transform(y_encoded)
print("디코딩된 레이블:", y_decoded)

원본 레이블:
 0        Relevant
1        Relevant
2        Relevant
3        Relevant
4        Relevant
           ...   
10871    Relevant
10872    Relevant
10873    Relevant
10874    Relevant
10875    Relevant
Name: choose_one, Length: 10860, dtype: object
인코딩된 레이블:
 [1 1 1 ... 1 1 1]
디코딩된 레이블: ['Relevant' 'Relevant' 'Relevant' ... 'Relevant' 'Relevant' 'Relevant']


In [67]:
X_encoded

NameError: name 'X_encoded' is not defined

In [69]:
len(y_encoded)

10860

In [45]:
# tokenize
# sequence padding

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# 학습된 토큰화기를 이용해 훈련 데이터와 테스트 데이터를 정수 시퀀스로 변환
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [46]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api_export_path',
 '_api_export_symbol_id',
 'analyzer',
 'char_level',
 'document_count',
 'filters',
 'fit_on_sequences',
 'fit_on_texts',
 'get_config',
 'index_docs',
 'index_word',
 'lower',
 'num_words',
 'oov_token',
 'sequences_to_matrix',
 'sequences_to_texts',
 'sequences_to_texts_generator',
 'split',
 'texts_to_matrix',
 'texts_to_sequences',
 'texts_to_sequences_generator',
 'to_json',
 'word_counts',
 'word_docs',
 'word_index']

In [47]:
tokenizer.word_index

{'fire': 1,
 'like': 2,
 'im': 3,
 'amp': 4,
 'get': 5,
 'u': 6,
 'new': 7,
 'via': 8,
 'one': 9,
 'news': 10,
 'dont': 11,
 'people': 12,
 'time': 13,
 'emergency': 14,
 'disaster': 15,
 'video': 16,
 'year': 17,
 'would': 18,
 'body': 19,
 'police': 20,
 'building': 21,
 'go': 22,
 'day': 23,
 'still': 24,
 'life': 25,
 'storm': 26,
 'home': 27,
 'say': 28,
 'crash': 29,
 'burning': 30,
 'attack': 31,
 'got': 32,
 'first': 33,
 'know': 34,
 'suicide': 35,
 'family': 36,
 'world': 37,
 'back': 38,
 'make': 39,
 'car': 40,
 'california': 41,
 'rt': 42,
 'train': 43,
 'flood': 44,
 'today': 45,
 'see': 46,
 'love': 47,
 'bomb': 48,
 'nuclear': 49,
 'death': 50,
 'going': 51,
 'hiroshima': 52,
 'full': 53,
 'look': 54,
 'watch': 55,
 'man': 56,
 'two': 57,
 'let': 58,
 'service': 59,
 'cant': 60,
 'take': 61,
 'pm': 62,
 'youtube': 63,
 'think': 64,
 'war': 65,
 'want': 66,
 'dead': 67,
 'last': 68,
 'need': 69,
 'accident': 70,
 'way': 71,
 'good': 72,
 'weapon': 73,
 'bombing': 74,
 'k

In [1]:
tokenizer.word_index[-1]

NameError: name 'tokenizer' is not defined

In [48]:
len(tokenizer.word_index)

22222

In [49]:
# check max len of sentences to cut if too long to optimize response time
max_len = max(len(x) for x in X_train)
max_len

137

In [50]:
# fill vacant word as 0 from preorder
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [51]:
# y label chagne to categorical (one-hot encoding)
y_train_seq = to_categorical(y_train_encoded)
y_test_seq = to_categorical(y_test_encoded)

In [55]:
y_test_seq

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [70]:
y_train_seq

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall  # 필요한 지표를 임포트


model_dnn = Sequential()  # will be layered
model_dnn.add(
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)
)  # add 1 to adjust padding
model_dnn.add(Dropout(0.2))  # prevent overfit
model_dnn.add(Flatten())
model_dnn.add(Dense(512, activation="relu"))  # add dense layers as you want
model_dnn.add(
    Dense(len(y_train_seq[0]), activation="sigmoid")
)  # final dense layer should be sigmoid when binary category, else softmax

model_dnn.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy", Precision(), Recall()],
)

model_dnn.fit(
    X_train_pad, y_train_seq, epochs=20, batch_size=32, validation_data=(X_test_pad, y_test_seq)
)

Epoch 1/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - accuracy: 0.5383 - loss: 0.7258 - precision_6: 0.5266 - recall_6: 0.3605 - val_accuracy: 0.7758 - val_loss: 0.4805 - val_precision_6: 0.7761 - val_recall_6: 0.7739
Epoch 2/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.8451 - loss: 0.3805 - precision_6: 0.8427 - recall_6: 0.8477 - val_accuracy: 0.8278 - val_loss: 0.4225 - val_precision_6: 0.8264 - val_recall_6: 0.8287
Epoch 3/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.9424 - loss: 0.1637 - precision_6: 0.9420 - recall_6: 0.9426 - val_accuracy: 0.8177 - val_loss: 0.5032 - val_precision_6: 0.8170 - val_recall_6: 0.8181
Epoch 4/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.9832 - loss: 0.0579 - precision_6: 0.9830 - recall_6: 0.9831 - val_accuracy: 0.8080 - val_loss: 0.5536 - val_precision_6: 0.8082 - val_recall_6: 0.8089
Epoch 5/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.9917 - loss: 0.0292 - precision_6: 0.9920 - recall_6: 0.9

### 출처:
https://blog.eduonix.com/internet-of-things/simple-nlp-based-chatbot-python/